In [15]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "acc0f6789b052a92716a21cbf9d2aa5a"  # Replace with your actual API key


provinces = {
    "Chiang Mai":{
        "lat": 14.0134,
        "lon": 98.6601
    },
    "Lamphun":{
        "lat": 18.5745,
        "lon": 99.0087
    },
    "Lampang":{
        "lat": 17.6289,
        "lon": 100.0977
    },
    "Uttaradit":{
        "lat": 17.6255,
        "lon": 100.0942
    },
    "Phrae":{
        "lat": 18.1459,
        "lon": 100.1410
    },
    "Nan":{
        "lat": 18.7838,
        "lon": 100.7789
    },
    "Phayao":{
        "lat": 19.1920,
        "lon": 99.8788
    },
    "Chiang Rai":{
        "lat": 19.9086 ,
        "lon": 99.8325
    },
    "Mae Hong Son":{
        "lat": 19.3006 ,
        "lon": 97.9686
}
}
# Function to fetch and process weather data
def get_weather_data(province='Pathum Thani'):
    
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        timestamp = datetime.fromtimestamp(data['dt'])
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'requested_province':province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [16]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            9 non-null      datetime64[ns]
 1   year                 9 non-null      int64         
 2   month                9 non-null      int64         
 3   day                  9 non-null      int64         
 4   hour                 9 non-null      int64         
 5   minute               9 non-null      int64         
 6   requested_province   9 non-null      object        
 7   location             9 non-null      object        
 8   weather_main         9 non-null      object        
 9   weather_description  9 non-null      object        
 10  main.temp            9 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(4)
memory usage: 924.0+ bytes


,timestamp,year,month,day,hour,minute,requested_province,location,weather_main,weather_description,main.temp
0,2025-04-28 17:53:12,2025,4,28,17,53,Chiang Mai,Dawei,Clouds,overcast clouds,24.52
1,2025-04-28 17:53:12,2025,4,28,17,53,Lamphun,Lamphun,Clouds,broken clouds,34.65
2,2025-04-28 17:53:13,2025,4,28,17,53,Lampang,Uttaradit,Clouds,broken clouds,31.10
3,2025-04-28 17:53:14,2025,4,28,17,53,Uttaradit,Uttaradit,Clouds,broken clouds,31.10
4,2025-04-28 17:53:15,2025,4,28,17,53,Phrae,Phrae,Clouds,few clouds,37.04


In [17]:
dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)
print(dt) 

2025-04-28 17:53:23.092132+06:42


In [18]:
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
#lakefs_endpoint = "http://lakefs-dev:8000/"
lakefs_endpoint = "http://localhost:8001/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

In [19]:
df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour']
)

In [20]:
from datetime import datetime
import pytz

dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)

In [21]:
print(dt)

2025-04-28 17:53:31.697772+06:42
